In [32]:
import sys
sys.path.append('/home/benkoziol/l/ocgis/src')
import ocgis
assert(ocgis.__release__ == '1.2.0.dev1')

Configure some environment variables to point to the head directory containing climate data files used in the demo as well as the output directory.

In [33]:
import tempfile
ocgis.env.DIR_DATA = '/home/benkoziol/data'
ocgis.env.DIR_OUTPUT = tempfile.mkdtemp()
print ocgis.env.DIR_OUTPUT

/tmp/tmp_WwO_r


Inspect a target file's metadata.

In [34]:
uri = 'tas_day_CanCM4_decadal2011_r2i1p1_20120101-20211231.nc'
variable = 'tas'
rd = ocgis.RequestDataset(uri=uri,variable=variable)
rd.inspect()

URI = /home/benkoziol/data/climate_data/CanCM4/tas_day_CanCM4_decadal2011_r2i1p1_20120101-20211231.nc
VARIABLE = tas

=== Temporal =============

Name = time
Count = 3650
Has Bounds = True
Data Type = float64
Start Date = 2012-01-01 00:00:00
End Date = 2022-01-01 00:00:00
Calendar = 365_day
Units = days since 1850-1-1
Resolution (Days) = 1

=== Spatial ==============

Spatial Reference = CFWGS84
Proj4 String = +proj=longlat +ellps=WGS84 +towgs84=0,0,0,0,0,0,0 +no_defs 
Extent = (-1.40625, -90.0, 358.59375, 90.0)
Geometry Interface = SpatialGeometryPolygonDimension
Resolution = 2.8125
Count = 8192

=== Level ================

No level dimension.

=== Metadata Dump ========

dimensions:
    time = ISUNLIMITED ; // 3650 currently
    lat = 64 ;
    lon = 128 ;
    bnds = 2 ;

variables:
    float64 time(time) ;
      time:bounds = "time_bnds" ;
      time:units = "days since 1850-1-1" ;
      time:calendar = "365_day" ;
      time:axis = "T" ;
      time:long_name = "time" ;
      time:st

Subset a target file by the boundary of California using an intersects GIS operation (the default), and write the data to an ESRI Shapefile. Select the first time coordinate only.

In [35]:
geom = '/home/benkoziol/l/project/ocg/bin/shp/state_boundaries/state_boundaries.shp'
ops = ocgis.OcgOperations(dataset=rd, geom=geom, geom_select_uid=[25], snippet=True,
                          output_format='shp', prefix='ca')
ops.execute()

'/tmp/tmp_WwO_r/ca/ca.shp'

 Also write the model grid to shapefile.

In [36]:
ocgis.OcgOperations(dataset=rd, output_format='shp', snippet=True, prefix='grid').execute()

'/tmp/tmp_WwO_r/grid/grid.shp'

Spatially average the grid cells clipped to the boundary of California for all the June, July, and August months in the target dataset. Write the output data to CSV.

In [37]:
import webbrowser
rd = ocgis.RequestDataset(uri=uri, variable=variable, time_region={'month': [6, 7, 8]})
ops = ocgis.OcgOperations(dataset=rd, geom=geom, geom_select_uid=[25], spatial_operation='clip',
                          output_format='csv', prefix='ca_spatial_average', aggregate=True)
ret = ops.execute()
print(ret)
webbrowser.open(ret)

/tmp/tmp_WwO_r/ca_spatial_average/ca_spatial_average.csv


True

Perform a difference calulation between two variables using a string function. Inspect the metadata of the output NetCDF file.

In [38]:
rd1 = ocgis.RequestDataset(uri='tasmax_day_CanCM4_decadal2010_r2i1p1_20110101-20201231.nc',
                           variable='tasmax')
rd2 = ocgis.RequestDataset(uri='tasmin_day_CanCM4_decadal2010_r2i1p1_20110101-20201231.nc',
                           variable='tasmin')
calc = 'diff=tasmax-tasmin'
ops = ocgis.OcgOperations(dataset=[rd1, rd2], calc=calc, output_format='nc', geom='state_boundaries',
                          select_ugid=[25], prefix='diff')
ret = ops.execute()
print(ocgis.Inspect(ret))

URI = /tmp/tmp_WwO_r/diff/diff.nc
VARIABLE = diff

=== Temporal =============

Name = time
Count = 3650
Has Bounds = True
Data Type = float64
Start Date = 2011-01-01 00:00:00
End Date = 2021-01-01 00:00:00
Calendar = 365_day
Units = days since 1850-1-1
Resolution (Days) = 1

=== Spatial ==============

Spatial Reference = CFWGS84
Proj4 String = +proj=longlat +ellps=WGS84 +towgs84=0,0,0,0,0,0,0 +no_defs 
Extent = (234.84375, 30.696654256231533, 246.09375, 44.64946315270447)
Geometry Interface = SpatialGeometryPolygonDimension
Resolution = 2.80153088965
Count = 20

=== Level ================

No level dimension.

=== Metadata Dump ========

dimensions:
    time = 3650 ;
    bnds = 2 ;
    lat = 5 ;
    lon = 4 ;

variables:
    float64 time(time) ;
      time:axis = "T" ;
      time:bounds = "time_bnds" ;
      time:units = "days since 1850-1-1" ;
      time:calendar = "365_day" ;
      time:long_name = "time" ;
      time:standard_name = "time" ;
    float64 time_bnds(time, bnds) ;
    

Calculate a sequence of statistics to produce a July time series conforming the target units from Kelvin to Celsius. Perform the calculations on the spatially averaged data for California.

In [39]:
import webbrowser
rd = ocgis.RequestDataset(uri=uri, variable=variable, time_region={'month': [7]}, conform_units_to='celsius', 
                          name='calcs')
calc = [{'func': 'mean', 'name': 'mean'},
        {'func': 'std', 'name': 'stdev'},
        {'func': 'min', 'name': 'min'},
        {'func': 'max', 'name': 'max'},
        {'func': 'median', 'name': 'median'},
        {'func': 'freq_perc', 'name': 'fp_95', 'kwds': {'percentile': 95.0}},
        {'func': 'freq_perc', 'name': 'fp_5', 'kwds':{'percentile': 5.0}},]
calc_grouping = ['month','year']
ops = ocgis.OcgOperations(dataset=rd, geom='state_boundaries', geom_select_uid=[25, 26], spatial_operation='clip',
                          output_format= 'csv', prefix='ca_calcs', aggregate=True, calc=calc,
                          calc_grouping=calc_grouping)
ret = ops.execute()
print(ret)
webbrowser.open(ret)

/tmp/tmp_WwO_r/ca_calcs/ca_calcs.csv


True

Perform the same operation returning the data as a "collection". Print the derived variable aliases.

In [41]:
ops.output_format = 'numpy'
ret = ops.execute()
print(ret)
print(ret[25][rd.name].variables.keys())

SpatialCollection([(25, OrderedDict([('calcs', <ocgis.interface.base.field.DerivedField object at 0x7f971164f490>)])), (26, OrderedDict([('calcs', <ocgis.interface.base.field.DerivedField object at 0x7f97119dc3d0>)]))])
['mean', 'stdev', 'min', 'max', 'median', 'fp_95', 'fp_5']


Variable values are always stored as five dimensions: realization, time, level, row, column

In [43]:
print(ret[25][rd.name].variables['mean'])
print(ret[25][rd.name].variables['mean'].value.shape)

DerivedVariable(name="mean", alias="mean", units="celsius")
(1, 10, 1, 1, 1)


Print some time values from the temporal dimension.

In [45]:
print(ret[25][rd.name].temporal.value_datetime)
print(ret[25][rd.name].temporal.bounds_datetime)

[datetime.datetime(2012, 7, 16, 0, 0) datetime.datetime(2013, 7, 16, 0, 0)
 datetime.datetime(2014, 7, 16, 0, 0) datetime.datetime(2015, 7, 16, 0, 0)
 datetime.datetime(2016, 7, 16, 0, 0) datetime.datetime(2017, 7, 16, 0, 0)
 datetime.datetime(2018, 7, 16, 0, 0) datetime.datetime(2019, 7, 16, 0, 0)
 datetime.datetime(2020, 7, 16, 0, 0) datetime.datetime(2021, 7, 16, 0, 0)]
[[datetime.datetime(2012, 7, 1, 0, 0) datetime.datetime(2012, 8, 1, 0, 0)]
 [datetime.datetime(2013, 7, 1, 0, 0) datetime.datetime(2013, 8, 1, 0, 0)]
 [datetime.datetime(2014, 7, 1, 0, 0) datetime.datetime(2014, 8, 1, 0, 0)]
 [datetime.datetime(2015, 7, 1, 0, 0) datetime.datetime(2015, 8, 1, 0, 0)]
 [datetime.datetime(2016, 7, 1, 0, 0) datetime.datetime(2016, 8, 1, 0, 0)]
 [datetime.datetime(2017, 7, 1, 0, 0) datetime.datetime(2017, 8, 1, 0, 0)]
 [datetime.datetime(2018, 7, 1, 0, 0) datetime.datetime(2018, 8, 1, 0, 0)]
 [datetime.datetime(2019, 7, 1, 0, 0) datetime.datetime(2019, 8, 1, 0, 0)]
 [datetime.datetime(2020

Print example variable values.

In [46]:
print(ret[25][rd.name].variables['mean'].value.squeeze())

[27.395915985107422 26.349559783935547 24.475839614868164 25.92029571533203
 26.61911964416504 23.823287963867188 25.99759864807129 25.82523536682129
 26.73784637451172 24.689342498779297]


Geometries are stored as Shapely objects with associated attributes.

In [47]:
print(type(ret.geoms[25]))
print(ret.geoms[25]).bounds
print(ret.properties[25].dtype.names)
print(ret.properties[25])

<class 'shapely.geometry.multipolygon.MultiPolygon'>
(-124.39263831223747, 32.53578135776605, -114.12523030267519, 42.00219136658233)
('UGID', 'STATE_FIPS', 'ID', 'STATE_NAME', 'STATE_ABBR')
[(25, '06', 25.0, 'California', 'CA')]


Read a data file with a coordinate system, and inspect its properties. Also take a closer look at some field properties.

In [49]:
uri = 'tas_RCM3_ncep_1981010103.nc'
# Variables can be auto-discovered when reasonable CF metadata is present.
rd = ocgis.RequestDataset(uri=uri)
field = rd.get()
assert(field.variables['tas']._value is None)
# Values are not loaded after slicing/subsetting.
sub = field[:, 0:10, :, 5, 8]
assert(sub.variables['tas']._value is None)
# The "source index" is sliced and used for on-demand loading.
print(sub.temporal._src_idx)

[0 1 2 3 4 5 6 7 8 9]


Take a closer look at the coordinate system object.

In [50]:
# The coordinate system objects are refined versions of PROJ.4 dictionaries and OSR spatial reference objects.
print(type(field.spatial.crs))
print(field.spatial.crs.value)
print(field.spatial.crs.sr.ExportToProj4())

<class 'ocgis.interface.base.crs.CFNarccapObliqueMercator'>
{'lonc': -97, 'ellps': 'WGS84', 'y_0': 3175000, 'no_defs': True, 'proj': 'omerc', 'x_0': 3925000, 'units': 'm', 'alpha': 360, 'k': 1, 'gamma': 360, 'lat_0': 47.5}
+proj=omerc +lat_0=47.5 +lonc=-97 +alpha=360 +k=1 +x_0=3925000 +y_0=3175000 +gamma=360 +ellps=WGS84 +units=m +no_defs 


For three variables, calculate monthly averages for the year 2014 for each U.S. state boundary.

In [51]:
rd1 = ocgis.RequestDataset(uri='tasmax_day_CanCM4_decadal2010_r2i1p1_20110101-20201231.nc')
rd2 = ocgis.RequestDataset(uri='tasmin_day_CanCM4_decadal2010_r2i1p1_20110101-20201231.nc')
rd3 = ocgis.RequestDataset(uri='tas_day_CanCM4_decadal2010_r2i1p1_20110101-20201231.nc')
calc = [{'func': 'mean', 'name': 'mean'}]
calc_grouping = ['month']
ops = ocgis.OcgOperations(dataset=[rd1, rd2, rd3], geom='state_boundaries', aggregate=True,
                          output_format='shp', spatial_operation='clip', prefix='temps',
                          calc=calc, calc_grouping=calc_grouping, time_region={'year': [2014]},
                          headers=['value', 'calc_alias', 'year', 'month', 'alias'], conform_units_to='fahrenheit')
ret = ops.execute()
print(ret)

/tmp/tmp_WwO_r/temps/temps.shp


Use ESMF regridding with a subset and spatial aggregation, writing the data to shapefile.

In [52]:
rd_src = ocgis.RequestDataset(uri='tas_day_CanCM4_decadal2010_r2i1p1_20110101-20201231.nc',
                              variable='tas')
rd_dest = ocgis.RequestDataset(uri='nldas_met_update.obs.daily.pr.1991.nc')
print rd_src.get().spatial.grid.resolution
print rd_dest.get().spatial.grid.resolution

regrid_options = {'with_corners': False}
ops = ocgis.OcgOperations(dataset=rd_src, regrid_destination=rd_dest, geom_select_uid=[6, 16], 
                          agg_selection=True, geom='state_boundaries', snippet=True,
                          output_format='shp', prefix='regrid', regrid_options=regrid_options)
print ops.execute()

2.8125
0.125
/tmp/tmp_WwO_r/regrid/regrid.shp


Shapefiles may also be used as fields along with ESMPy fields (almost)!

In [53]:
rd = ocgis.RequestDataset(uri='/home/benkoziol/l/project/ocg/bin/shp/state_boundaries/state_boundaries.shp')
sfield = rd.get()
print(sfield.variables.keys())
print(sfield.variables['STATE_NAME'].value.squeeze())

[u'UGID', u'STATE_FIPS', u'ID', u'STATE_NAME', u'STATE_ABBR']
['Hawaii' 'Washington' 'Montana' 'Maine' 'North Dakota' 'South Dakota'
 'Wyoming' 'Wisconsin' 'Idaho' 'Vermont' 'Minnesota' 'Oregon'
 'New Hampshire' 'Iowa' 'Massachusetts' 'Nebraska' 'New York'
 'Pennsylvania' 'Connecticut' 'Rhode Island' 'New Jersey' 'Indiana'
 'Nevada' 'Utah' 'California' 'Ohio' 'Illinois' 'District of Columbia'
 'Delaware' 'West Virginia' 'Maryland' 'Colorado' 'Kentucky' 'Kansas'
 'Virginia' 'Missouri' 'Arizona' 'Oklahoma' 'North Carolina' 'Tennessee'
 'Texas' 'New Mexico' 'Alabama' 'Mississippi' 'Georgia' 'South Carolina'
 'Arkansas' 'Louisiana' 'Florida' 'Michigan' 'Alaska']
